# Employee Churn Clasifier Project

### First we are going to import the libraries that we will use in the project

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import traceback
import re
import string
import pandas.core.algorithms as algos
from pandas import Series
import os
import datetime as dt
import matplotlib as matplot
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
'''First we are going to upload the information as a pandas dataframe
As a best practice, we are going to upload an extract from the file in order to get the columns data types, moreover
in big data sets, you have to specify the data types to optimize memory'''
# The filepath should be update in order to execute this jupyter notebook.
data=pd.read_csv("Documents/HRDataset_v13.csv",nrows=150,encoding='latin')
Dictionary=data.dtypes.to_dict()

In [3]:
#We have an overview of the first 5 rows of the dataset
data.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,PayRate,...,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30
0,"Brown, Mia",1103024456,1,1,0,1,1,3,1,28.50,...,Admin Offices,Brandon R. LeBlanc,1,Diversity Job Fair,Fully Meets,2.04,2,6,1/15/2019,0.0
1,"LaRotonda, William",1106026572,0,2,1,1,1,3,0,23.00,...,Admin Offices,Brandon R. LeBlanc,1,Website Banner Ads,Fully Meets,5.00,4,4,1/17/2019,0.0
2,"Steans, Tyrone",1302053333,0,0,1,1,1,3,0,29.00,...,Admin Offices,Brandon R. LeBlanc,1,Internet Search,Fully Meets,3.90,5,5,1/18/2019,0.0
3,"Howard, Estelle",1211050782,1,1,0,1,1,3,0,21.50,...,Admin Offices,Brandon R. LeBlanc,1,Pay Per Click - Google,Fully Meets,3.24,3,4,NaN,NaN
4,"Singh, Nan",1307059817,0,0,0,1,1,3,0,16.56,...,Admin Offices,Brandon R. LeBlanc,1,Website Banner Ads,Fully Meets,5.00,3,5,1/15/2019,0.0


In [4]:
#This is the shape of the dataframe before uploading all the rows
data.shape

(150, 35)

In [5]:
#We have to change the Manager ID type from int to float since it has NA values and pandas does not support NA values in int types
Dictionary['ManagerID']="float64"

In [6]:
#Here we upload all the dataset withe the dictionary dtypes
data=pd.read_csv("Documents/HRDataset_v13.csv",dtype=Dictionary,encoding='latin')

In [7]:
#This is the new shape of the dataframe after uploading all the csv file.
data.shape

(310, 35)

In [8]:
#Here we are going to compute the age of the employee
data['Age']=dt.date.today().year-pd.to_datetime(data['DateOfBirth']).dt.year
data.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,PayRate,...,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Age
0,"Brown, Mia",1103024456,1,1,0,1,1,3,1,28.50,...,Brandon R. LeBlanc,1.0,Diversity Job Fair,Fully Meets,2.04,2,6,1/15/2019,0.0,33
1,"LaRotonda, William",1106026572,0,2,1,1,1,3,0,23.00,...,Brandon R. LeBlanc,1.0,Website Banner Ads,Fully Meets,5.00,4,4,1/17/2019,0.0,36
2,"Steans, Tyrone",1302053333,0,0,1,1,1,3,0,29.00,...,Brandon R. LeBlanc,1.0,Internet Search,Fully Meets,3.90,5,5,1/18/2019,0.0,34
3,"Howard, Estelle",1211050782,1,1,0,1,1,3,0,21.50,...,Brandon R. LeBlanc,1.0,Pay Per Click - Google,Fully Meets,3.24,3,4,NaN,NaN,35
4,"Singh, Nan",1307059817,0,0,0,1,1,3,0,16.56,...,Brandon R. LeBlanc,1.0,Website Banner Ads,Fully Meets,5.00,3,5,1/15/2019,0.0,32


### We are going to quit the Terminated for Cause  since our approach is limited to Voluntary Terminated

In [9]:
data=data[~data['EmploymentStatus'].isin(["Terminated for Cause"])]

In [10]:
#Here we compute the Anitquity of the Employee, since the Hire Date to the Termination Date or Today. It will depende if the
#employee left or not the company.
data['Antiquity']=np.where(data['Termd']==1,pd.to_datetime(data['DateofTermination']).dt.year-\
pd.to_datetime(data['DateofHire']).dt.year,dt.date.today().year-pd.to_datetime(data['DateofHire']).dt.year)                          

data.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,PayRate,...,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Age,Antiquity
0,"Brown, Mia",1103024456,1,1,0,1,1,3,1,28.50,...,1.0,Diversity Job Fair,Fully Meets,2.04,2,6,1/15/2019,0.0,33,12.0
1,"LaRotonda, William",1106026572,0,2,1,1,1,3,0,23.00,...,1.0,Website Banner Ads,Fully Meets,5.00,4,4,1/17/2019,0.0,36,6.0
2,"Steans, Tyrone",1302053333,0,0,1,1,1,3,0,29.00,...,1.0,Internet Search,Fully Meets,3.90,5,5,1/18/2019,0.0,34,6.0
4,"Singh, Nan",1307059817,0,0,0,1,1,3,0,16.56,...,1.0,Website Banner Ads,Fully Meets,5.00,3,5,1/15/2019,0.0,32,5.0
5,"Smith, Leigh Ann",711007713,1,1,0,5,1,3,1,20.50,...,1.0,Diversity Job Fair,Fully Meets,3.80,4,4,NaN,NaN,33,2.0


In [11]:
data.columns

Index(['Employee_Name', 'EmpID', 'MarriedID', 'MaritalStatusID', 'GenderID',
       'EmpStatusID', 'DeptID', 'PerfScoreID', 'FromDiversityJobFairID',
       'PayRate', 'Termd', 'PositionID', 'Position', 'State', 'Zip',
       'DateOfBirth', 'Sex', 'MaritalDesc', 'CitizenDesc', 'HispanicLatino',
       'RaceDesc', 'DateofHire', 'DateofTermination', 'TermReason',
       'EmploymentStatus', 'Department', 'ManagerName', 'ManagerID',
       'RecruitmentSource', 'PerformanceScore', 'EngagementSurvey',
       'EmpSatisfaction', 'SpecialProjectsCount', 'LastPerformanceReview_Date',
       'DaysLateLast30', 'Age', 'Antiquity'],
      dtype='object')

In [12]:
#We have done an exploratory analysis with Tableau,and we are going to explore the next variables, Antiquitiy, Age, 
#Recruitment Source,Performance, Department,Position,Manager Name and Pay Rate
variables=['Position','PayRate','RecruitmentSource','MaritalDesc','Department','ManagerName','PerformanceScore',\
           'Antiquity','Age','Termd']
data=data.loc[:, data.columns.isin(variables)]
data.head()


,PayRate,Termd,Position,MaritalDesc,Department,ManagerName,RecruitmentSource,PerformanceScore,Age,Antiquity
0,28.50,0,Accountant I,Married,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,12.0
1,23.00,0,Accountant I,Divorced,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,36,6.0
2,29.00,0,Accountant I,Single,Admin Offices,Brandon R. LeBlanc,Internet Search,Fully Meets,34,6.0
4,16.56,0,Administrative Assistant,Single,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,32,5.0
5,20.50,1,Administrative Assistant,Married,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,2.0


In [13]:
#This code turns the continuous variables into discrete variables,and make bins of them in order to apply the information value
#metric which will tell us. What is the impact of each variable for forecasting the dependent variable in this case termd
max_bin = 20
force_bin = 3
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE',\
             'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE',\
             'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [14]:
#Here the code show us which is the information value of each variable, we can see that the best variable to predict the outcome
#is the Antiquity, such as we have seen in the exploratory analysis where all the employees which left the company where does 
#with less than 6 years in the company. The recruitment source, in the exploratory analysis we see that the Employee Referal was
#the best recruitment in contrast Facebook and Google,Bing and Yahoo where bad sources. The manager name we have seen that 
#Webster, Amy and Kissiy Sullivan hold the worst churn rate of employees. Department we have seen that the production department
#it is the biggest deparment but also is the department with more rotation (nearly 40%).The position is related with the department
#Marital Desc we have seen that people that suffered in personal relationship tend to search for a change.


final_ivTotal, IVTotal = data_vars(data ,data.Termd)
IVTotal.sort_values('IV')

C:\Users\jjuarezp\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,VAR_NAME,IV
6,PerformanceScore,0.044580
5,PayRate,0.153572
0,Age,0.207734
4,MaritalDesc,0.274864
7,Position,0.320270
2,Department,0.353957
3,ManagerName,0.537467
8,RecruitmentSource,0.868741
1,Antiquity,3.243910


In [15]:
final_ivTotal[final_ivTotal['VAR_NAME']=='Antiquity']

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
111,Antiquity,0,3,76,70,0.921053,6,0.078947,0.795455,0.028986,3.312118,3.24391
112,Antiquity,4,6,114,17,0.149123,97,0.850877,0.193182,0.468599,-0.886116,3.24391
113,Antiquity,7,7,38,1,0.026316,37,0.973684,0.011364,0.178744,-2.755536,3.24391
114,Antiquity,8,14,67,0,0.000000,67,1.000000,0.000000,0.323671,0.000000,3.24391


In [16]:
final_ivTotal[final_ivTotal['VAR_NAME']=='RecruitmentSource'].sort_values('EVENT',ascending=False).head(7)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
68,RecruitmentSource,Diversity Job Fair,Diversity Job Fair,29,16,0.551724,13,0.448276,0.181818,0.062802,1.063021,0.868741
83,RecruitmentSource,Search Engine - Google Bing Yahoo,Search Engine - Google Bing Yahoo,24,14,0.583333,10,0.416667,0.159091,0.048309,1.191854,0.868741
75,RecruitmentSource,Monster.com,Monster.com,23,10,0.434783,13,0.565217,0.113636,0.062802,0.593018,0.868741
84,RecruitmentSource,Social Networks - Facebook Twitter etc,Social Networks - Facebook Twitter etc,9,6,0.666667,3,0.333333,0.068182,0.014493,1.548529,0.868741
76,RecruitmentSource,Newspager/Magazine,Newspager/Magazine,18,5,0.277778,13,0.722222,0.056818,0.062802,-0.100129,0.868741
87,RecruitmentSource,Word of Mouth,Word of Mouth,11,5,0.454545,6,0.545455,0.056818,0.028986,0.673060,0.868741
65,RecruitmentSource,Billboard,Billboard,15,4,0.266667,11,0.733333,0.045455,0.053140,-0.156219,0.868741


In [17]:
final_ivTotal[final_ivTotal['VAR_NAME']=='ManagerName'].sort_values('EVENT',ascending=False).head(7)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
64,ManagerName,Webster Butler,Webster Butler,21,13,0.619048,8,0.380952,0.147727,0.038647,1.340890,0.537467
45,ManagerName,Amy Dunn,Amy Dunn,21,13,0.619048,8,0.380952,0.147727,0.038647,1.340890,0.537467
59,ManagerName,Kissy Sullivan,Kissy Sullivan,20,10,0.500000,10,0.500000,0.113636,0.048309,0.855382,0.537467
61,ManagerName,Michael Albert,Michael Albert,21,8,0.380952,13,0.619048,0.090909,0.062802,0.369874,0.537467
52,ManagerName,Elijiah Gray,Elijiah Gray,21,7,0.333333,14,0.666667,0.079545,0.067633,0.162235,0.537467
48,ManagerName,Brannon Miller,Brannon Miller,21,6,0.285714,15,0.714286,0.068182,0.072464,-0.060909,0.537467
50,ManagerName,David Stanley,David Stanley,21,6,0.285714,15,0.714286,0.068182,0.072464,-0.060909,0.537467


In [18]:
final_ivTotal[final_ivTotal['VAR_NAME']=='Position'].sort_values('EVENT',ascending=False).head(7)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
23,Position,Production Technician I,Production Technician I,129,45,0.348837,84,0.651163,0.511364,0.405797,0.231228,0.32027
24,Position,Production Technician II,Production Technician II,57,26,0.456140,31,0.543860,0.295455,0.149758,0.679491,0.32027
22,Position,Production Manager,Production Manager,13,4,0.307692,9,0.692308,0.045455,0.043478,0.044452,0.32027
4,Position,Area Sales Manager,Area Sales Manager,26,2,0.076923,24,0.923077,0.022727,0.115942,-1.629525,0.32027
28,Position,Software Engineer,Software Engineer,8,2,0.250000,6,0.750000,0.022727,0.028986,-0.243230,0.32027
27,Position,Shared Services Manager,Shared Services Manager,2,1,0.500000,1,0.500000,0.011364,0.004831,0.855382,0.32027
25,Position,Sales Manager,Sales Manager,3,1,0.333333,2,0.666667,0.011364,0.009662,0.162235,0.32027


In [19]:
final_ivTotal[final_ivTotal['VAR_NAME']=='Department'].sort_values('EVENT',ascending=False)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
41,Department,Production,Production,200,75,0.375000,125,0.625000,0.852273,0.603865,0.344556,0.353957
40,Department,IT/IS,IT/IS,46,6,0.130435,40,0.869565,0.068182,0.193237,-1.041738,0.353957
42,Department,Sales,Sales,30,3,0.100000,27,0.900000,0.034091,0.130435,-1.341843,0.353957
38,Department,Admin Offices,Admin Offices,9,2,0.222222,7,0.777778,0.022727,0.033816,-0.397381,0.353957
43,Department,Software Engineering,Software Engineering,9,2,0.222222,7,0.777778,0.022727,0.033816,-0.397381,0.353957
39,Department,Executive Office,Executive Office,1,0,0.000000,1,1.000000,0.000000,0.004831,0.000000,0.353957


In [20]:
final_ivTotal[final_ivTotal['VAR_NAME']=='MaritalDesc'].sort_values('EVENT_RATE',ascending=False)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
33,MaritalDesc,Divorced,Divorced,30,16,0.533333,14,0.466667,0.181818,0.067633,0.988913,0.274864
37,MaritalDesc,Widowed,Widowed,7,3,0.428571,4,0.571429,0.034091,0.019324,0.567700,0.274864
34,MaritalDesc,Married,Married,117,40,0.341880,77,0.658120,0.454545,0.371981,0.200456,0.274864
36,MaritalDesc,Single,Single,129,28,0.217054,101,0.782946,0.318182,0.487923,-0.427534,0.274864
35,MaritalDesc,Separated,Separated,12,1,0.083333,11,0.916667,0.011364,0.053140,-1.542513,0.274864


In [21]:
final_ivTotal[final_ivTotal['VAR_NAME']=='PayRate'].sort_values('EVENT_RATE',ascending=False)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,PayRate,14,24,151,57,0.377483,94,0.622517,0.647727,0.454106,0.355138,0.153572
1,PayRate,24.25,80,144,31,0.215278,113,0.784722,0.352273,0.545894,-0.438019,0.153572


In [22]:
final_ivTotal[final_ivTotal['VAR_NAME']=='Age'].sort_values('EVENT_RATE',ascending=False).head(7)

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
110,Age,59,69,14,8,0.571429,6,0.428571,0.090909,0.028986,1.143064,0.207734
106,Age,47,48,15,7,0.466667,8,0.533333,0.079545,0.038647,0.721851,0.207734
105,Age,46,46,12,5,0.416667,7,0.583333,0.056818,0.033816,0.518910,0.207734
97,Age,36,36,16,6,0.375000,10,0.625000,0.068182,0.048309,0.344556,0.207734
104,Age,44,45,18,6,0.333333,12,0.666667,0.068182,0.057971,0.162235,0.207734
94,Age,33,33,18,6,0.333333,12,0.666667,0.068182,0.057971,0.162235,0.207734
107,Age,50,51,19,6,0.315789,13,0.684211,0.068182,0.062802,0.082192,0.207734


In [23]:
# Now we are going to train three different classification models,  first a logistic regression, then a SVM and finally
#a Random Forest
logistic=data
logistic.head()

,PayRate,Termd,Position,MaritalDesc,Department,ManagerName,RecruitmentSource,PerformanceScore,Age,Antiquity
0,28.50,0,Accountant I,Married,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,12.0
1,23.00,0,Accountant I,Divorced,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,36,6.0
2,29.00,0,Accountant I,Single,Admin Offices,Brandon R. LeBlanc,Internet Search,Fully Meets,34,6.0
4,16.56,0,Administrative Assistant,Single,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,32,5.0
5,20.50,1,Administrative Assistant,Married,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,2.0


### First we are going to create dummy variables for the categorical variables. Moreover, we are going to dismiss one of the variables to prevent colinearity

In [24]:
dummies = pd.get_dummies(logistic['Position']).rename(columns=lambda x: 'Position_' + str(x))
logistic = pd.concat([logistic, dummies], axis=1)
logistic.drop(['Position'],axis=1,inplace=True)
logistic.head()

,PayRate,Termd,MaritalDesc,Department,ManagerName,RecruitmentSource,PerformanceScore,Age,Antiquity,Position_Accountant I,...,Position_Production Technician I,Position_Production Technician II,Position_Sales Manager,Position_Senior BI Developer,Position_Shared Services Manager,Position_Software Engineer,Position_Software Engineering Manager,Position_Sr. Accountant,Position_Sr. DBA,Position_Sr. Network Engineer
0,28.50,0,Married,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,12.0,1,...,0,0,0,0,0,0,0,0,0,0
1,23.00,0,Divorced,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,36,6.0,1,...,0,0,0,0,0,0,0,0,0,0
2,29.00,0,Single,Admin Offices,Brandon R. LeBlanc,Internet Search,Fully Meets,34,6.0,1,...,0,0,0,0,0,0,0,0,0,0
4,16.56,0,Single,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,32,5.0,0,...,0,0,0,0,0,0,0,0,0,0
5,20.50,1,Married,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,2.0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
dummies = pd.get_dummies(logistic['MaritalDesc']).rename(columns=lambda x: 'MaritalDesc_' + str(x))
logistic = pd.concat([logistic, dummies], axis=1)
logistic.drop(['MaritalDesc'],axis=1,inplace=True)
logistic.head()

,PayRate,Termd,Department,ManagerName,RecruitmentSource,PerformanceScore,Age,Antiquity,Position_Accountant I,Position_Administrative Assistant,...,Position_Software Engineer,Position_Software Engineering Manager,Position_Sr. Accountant,Position_Sr. DBA,Position_Sr. Network Engineer,MaritalDesc_Divorced,MaritalDesc_Married,MaritalDesc_Separated,MaritalDesc_Single,MaritalDesc_Widowed
0,28.50,0,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,12.0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,23.00,0,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,36,6.0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,29.00,0,Admin Offices,Brandon R. LeBlanc,Internet Search,Fully Meets,34,6.0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,16.56,0,Admin Offices,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,32,5.0,0,1,...,0,0,0,0,0,0,0,0,1,0
5,20.50,1,Admin Offices,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,2.0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [26]:
dummies = pd.get_dummies(logistic['Department']).rename(columns=lambda x: 'Department_' + str(x))
logistic = pd.concat([logistic, dummies], axis=1)
logistic.drop(['Department'],axis=1,inplace=True)
logistic.head()

,PayRate,Termd,ManagerName,RecruitmentSource,PerformanceScore,Age,Antiquity,Position_Accountant I,Position_Administrative Assistant,Position_Area Sales Manager,...,MaritalDesc_Married,MaritalDesc_Separated,MaritalDesc_Single,MaritalDesc_Widowed,Department_Admin Offices,Department_Executive Office,Department_IT/IS,Department_Production,Department_Sales,Department_Software Engineering
0,28.50,0,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,12.0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
1,23.00,0,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,36,6.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,29.00,0,Brandon R. LeBlanc,Internet Search,Fully Meets,34,6.0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
4,16.56,0,Brandon R. LeBlanc,Website Banner Ads,Fully Meets,32,5.0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
5,20.50,1,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets,33,2.0,0,1,0,...,1,0,0,0,1,0,0,0,0,0


In [27]:
dummies = pd.get_dummies(logistic['ManagerName']).rename(columns=lambda x: 'ManagerName_' + str(x))
logistic = pd.concat([logistic, dummies], axis=1)
logistic.drop(['ManagerName'],axis=1,inplace=True)
logistic.head()

,PayRate,Termd,RecruitmentSource,PerformanceScore,Age,Antiquity,Position_Accountant I,Position_Administrative Assistant,Position_Area Sales Manager,Position_BI Developer,...,ManagerName_Jennifer Zamora,ManagerName_John Smith,ManagerName_Kelley Spirea,ManagerName_Ketsia Liebig,ManagerName_Kissy Sullivan,ManagerName_Lynn Daneault,ManagerName_Michael Albert,ManagerName_Peter Monroe,ManagerName_Simon Roup,ManagerName_Webster Butler
0,28.50,0,Diversity Job Fair,Fully Meets,33,12.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,23.00,0,Website Banner Ads,Fully Meets,36,6.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,29.00,0,Internet Search,Fully Meets,34,6.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16.56,0,Website Banner Ads,Fully Meets,32,5.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,20.50,1,Diversity Job Fair,Fully Meets,33,2.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
dummies = pd.get_dummies(logistic['RecruitmentSource']).rename(columns=lambda x: 'RecruitmentSource_' + str(x))
logistic = pd.concat([logistic, dummies], axis=1)
logistic.drop(['RecruitmentSource'],axis=1,inplace=True)
logistic.head()

,PayRate,Termd,PerformanceScore,Age,Antiquity,Position_Accountant I,Position_Administrative Assistant,Position_Area Sales Manager,Position_BI Developer,Position_BI Director,...,RecruitmentSource_On-line Web application,RecruitmentSource_Other,RecruitmentSource_Pay Per Click,RecruitmentSource_Pay Per Click - Google,RecruitmentSource_Professional Society,RecruitmentSource_Search Engine - Google Bing Yahoo,RecruitmentSource_Social Networks - Facebook Twitter etc,RecruitmentSource_Vendor Referral,RecruitmentSource_Website Banner Ads,RecruitmentSource_Word of Mouth
0,28.50,0,Fully Meets,33,12.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,23.00,0,Fully Meets,36,6.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,29.00,0,Fully Meets,34,6.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16.56,0,Fully Meets,32,5.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,20.50,1,Fully Meets,33,2.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
dummies = pd.get_dummies(logistic['PerformanceScore']).rename(columns=lambda x: 'PerformanceScore_' + str(x))
logistic = pd.concat([logistic, dummies], axis=1)
logistic.drop(['PerformanceScore'],axis=1,inplace=True)
logistic.head()

,PayRate,Termd,Age,Antiquity,Position_Accountant I,Position_Administrative Assistant,Position_Area Sales Manager,Position_BI Developer,Position_BI Director,Position_CIO,...,RecruitmentSource_Professional Society,RecruitmentSource_Search Engine - Google Bing Yahoo,RecruitmentSource_Social Networks - Facebook Twitter etc,RecruitmentSource_Vendor Referral,RecruitmentSource_Website Banner Ads,RecruitmentSource_Word of Mouth,PerformanceScore_Exceeds,PerformanceScore_Fully Meets,PerformanceScore_Needs Improvement,PerformanceScore_PIP
0,28.50,0,33,12.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,23.00,0,36,6.0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,29.00,0,34,6.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,16.56,0,32,5.0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
5,20.50,1,33,2.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [30]:
logistic.drop(['Position_Senior BI Developer','MaritalDesc_Separated','Department_Sales'\
                 ,'ManagerName_Simon Roup','RecruitmentSource_Vendor Referral','PerformanceScore_Exceeds'],axis=1,inplace=True)

In [31]:
logistic.head()

,PayRate,Termd,Age,Antiquity,Position_Accountant I,Position_Administrative Assistant,Position_Area Sales Manager,Position_BI Developer,Position_BI Director,Position_CIO,...,RecruitmentSource_Pay Per Click,RecruitmentSource_Pay Per Click - Google,RecruitmentSource_Professional Society,RecruitmentSource_Search Engine - Google Bing Yahoo,RecruitmentSource_Social Networks - Facebook Twitter etc,RecruitmentSource_Website Banner Ads,RecruitmentSource_Word of Mouth,PerformanceScore_Fully Meets,PerformanceScore_Needs Improvement,PerformanceScore_PIP
0,28.50,0,33,12.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,23.00,0,36,6.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,29.00,0,34,6.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,16.56,0,32,5.0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
5,20.50,1,33,2.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### First we are going to start with the logistic regression model

In [43]:
# I import the necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
#Here we separate the Daframe in dependent (Y) and independent (X) variables
X = logistic.loc[:, logistic.columns != 'Termd']
y = logistic.loc[:, logistic.columns == 'Termd']
#I split the dataframe in train/test in order to evaluate the classifier performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
#Here We call the logistic Regression Classifier, and we are going to set the random state=42 for all the classifiers
logreg = LogisticRegression(random_state=42,max_iter=3000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
#Here we evaluate the performance of the classifiers, the first one evaluate the mean accuracy in the prediction
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#The Log loss score is better when the value is low since it means the -log likelihood is better.
print('Log Loss of logistic regression classifier on test set: {:.2f}'.format(log_loss(y_test, y_pred)))
#The brier score model measures the accuracy of probabilistic predictions
print('Brier Score Loss of logistic regression classifier on test set: {:.2f}'.format(brier_score_loss(y_test, y_pred)))

Accuracy of logistic regression classifier on test set: 0.96
Log Loss of logistic regression classifier on test set: 1.55
Brier Score Loss of logistic regression classifier on test set: 0.04


C:\Users\jjuarezp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


### Here we have the coefficients of the logistic regression, first the variables which fosters the employee churn and next the variables which supress employee churn

In [33]:
# A caveat to take into account is that correlation does not imply causation. Even if the Manager Janet King appears like she
#fosterds the churn ratio, probably there are another root causes that help the churn of employees.

coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(logreg.coef_))], axis = 1)
coefficients.columns=['Variable','Coefficient']
coefficients['Absolute']=coefficients['Coefficient'].abs()
coefficients.sort_values('Coefficient',ascending=False).head(10)

,Variable,Coefficient,Absolute
72,RecruitmentSource_Monster.com,1.026314,1.026314
65,RecruitmentSource_Diversity Job Fair,0.971684,0.971684
70,RecruitmentSource_Internet Search,0.815881,0.815881
23,Position_Production Manager,0.561449,0.561449
40,Department_Production,0.559949,0.559949
52,ManagerName_Janet King,0.525834,0.525834
43,ManagerName_Amy Dunn,0.517227,0.517227
67,RecruitmentSource_Glassdoor,0.449915,0.449915
81,RecruitmentSource_Social Networks - Facebook T...,0.418074,0.418074
26,Position_Sales Manager,0.363357,0.363357


In [34]:
coefficients.sort_values('Coefficient',ascending=False).tail(10)

,Variable,Coefficient,Absolute
73,RecruitmentSource_Newspager/Magazine,-0.335760,0.335760
6,Position_BI Developer,-0.344462,0.344462
66,RecruitmentSource_Employee Referral,-0.386283,0.386283
78,RecruitmentSource_Pay Per Click - Google,-0.389600,0.389600
55,ManagerName_Kelley Spirea,-0.420622,0.420622
79,RecruitmentSource_Professional Society,-0.506965,0.506965
68,RecruitmentSource_Indeed,-0.692123,0.692123
47,ManagerName_Brian Champaigne,-0.692123,0.692123
39,Department_IT/IS,-0.814157,0.814157
2,Antiquity,-1.792807,1.792807


### Now we are going to use the Support Vector Machine(Using Kernel rbf)

In [35]:
#We import the library
from sklearn.svm import SVC
# Here we set the Vector Machine equal to the logistic Data frame.
VectorMachine=logistic
# Termination is the label we want to predict, it is important to mention that from this point on we are going to use numpy 
#arrays as input to the models, since these type of array works better for linear algebra operations.
Termination = np.array(VectorMachine['Termd'])
# Remove the labels from the features
# axis 1 refers to the columns
features= VectorMachine.drop('Termd', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)
#Here we split the data set again between training and test
train_features, test_features, train_Termination, test_Termination = train_test_split(features, Termination, \
test_size = 0.25, random_state = 42)
clf = SVC(gamma='auto',random_state=42)
clf.fit(train_features, train_Termination)
Termination_pred = clf.predict(test_features)
print('Accuracy of SCV-kernel classifier on test set: {:.2f}'.format(clf.score(test_features, test_Termination)))
print('Log Loss of SCV-kerne classifier on test set: {:.2f}'.format(log_loss(test_Termination, Termination_pred)))
print('Brier Score Loss of SCV-kernel classifier on test set: {:.2f}'.format(brier_score_loss(test_Termination,\
Termination_pred)))

Accuracy of SCV-kernel classifier on test set: 0.84
Log Loss of SCV-kerne classifier on test set: 5.60
Brier Score Loss of SCV-kernel classifier on test set: 0.16


### Now we are going to use the Linear Support Vector Machine

In [42]:
#Here we import the library
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=42,max_iter=5000)
#We are going to use the same training and test sets that we use in SVC kernel rbf
clf.fit(train_features, train_Termination)
Termination_pred = clf.predict(test_features)
print('Accuracy of Linear SCV classifier on test set: {:.2f}'.format(clf.score(test_features, test_Termination)))
print('Log Loss of LinearSVC classifier on test set: {:.2f}'.format(log_loss(test_Termination, Termination_pred)))
print('Brier Score Loss of Linear SVC classifier on test set: {:.2f}'.format(brier_score_loss(test_Termination,\
Termination_pred)))

Accuracy of Linear SCV classifier on test set: 0.92
Log Loss of LinearSVC classifier on test set: 2.80
Brier Score Loss of Linear SVC classifier on test set: 0.08


C:\Users\jjuarezp\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### Here we have the coefficients of the logistic regression, first the variables which fosters the employee churn and next the variables which supress employee churn

In [37]:
# A caveat to take into account is that correlation does not imply causation. Even if the Manager Janet King appears like she
#fosterds the churn ratio, probably there are another root causes that help the churn of employees.
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(clf.coef_))], axis = 1)
coefficients.columns=['Variable','Coefficient']
coefficients['Absolute']=coefficients['Coefficient'].abs()
coefficients.sort_values('Coefficient',ascending=False).head(10)

,Variable,Coefficient,Absolute
24,Position_Production Technician I,0.558927,0.558927
33,MaritalDesc_Divorced,0.553087,0.553087
65,RecruitmentSource_Diversity Job Fair,0.532292,0.532292
50,ManagerName_Elijiah Gray,0.418632,0.418632
71,RecruitmentSource_MBTA ads,0.364422,0.364422
72,RecruitmentSource_Monster.com,0.330389,0.330389
40,Department_Production,0.328093,0.328093
69,RecruitmentSource_Information Session,0.276913,0.276913
81,RecruitmentSource_Social Networks - Facebook T...,0.189362,0.189362
28,Position_Software Engineer,0.183199,0.183199


In [38]:
coefficients.sort_values('Coefficient',ascending=False).tail(10)

,Variable,Coefficient,Absolute
25,Position_Production Technician II,-0.124597,0.124597
73,RecruitmentSource_Newspager/Magazine,-0.129242,0.129242
66,RecruitmentSource_Employee Referral,-0.137224,0.137224
82,RecruitmentSource_Website Banner Ads,-0.197904,0.197904
35,MaritalDesc_Single,-0.247790,0.247790
59,ManagerName_Michael Albert,-0.279424,0.279424
68,RecruitmentSource_Indeed,-0.341126,0.341126
47,ManagerName_Brian Champaigne,-0.341126,0.341126
39,Department_IT/IS,-0.355653,0.355653
2,Antiquity,-0.748147,0.748147


### Finally we are going to test the random forest classifier

In [44]:
#This method is different from the before classifier since this is not a linear classifier, this method takes a bootstrap and 
#fits different tree classifiers to the data, and select the best tree classifier among the rest
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 250 decision trees
rf= RandomForestClassifier(n_estimators = 1000,random_state=42)
# Train the model on training data
rf.fit(train_features, train_Termination)
# Use the forest's predict method on the test data
Termination_pred = rf.predict(test_features)
print('Accuracy of Random Forest classifier on test set: {:.2f}'.format(rf.score(test_features, test_Termination)))
print('Log Loss of Random Forest classifier on test set: {:.2f}'.format(log_loss(test_Termination, Termination_pred)))
print('Brier Score Loss of Random Forest classifier on test set: {:.2f}'.format(brier_score_loss(test_Termination,\
Termination_pred)))

Accuracy of Random Forest classifier on test set: 0.91
Log Loss of Random Forest classifier on test set: 3.27
Brier Score Loss of Random Forest classifier on test set: 0.09


## The best classifiers is the logistic regression, it has the best metrics.